In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

zip_id = '1upOcakmV4Ym2sv-5K3MY88rA_QNzTXJV'
print ("Downloading zip file")
myzip = drive.CreateFile({'id': zip_id})
myzip.GetContentFile('model.zip')
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('model.zip', 'r')
zip_ref.extractall("breast_density_dataset")
zip_ref.close()

Uncompressing zip file


In [0]:
train = pd.read_csv("breast_density_dataset/train.csv")
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in train.columns[2:-1]:
  train[col]=le.fit_transform(train[col])
train.to_csv("breast_density_dataset/train.csv",index=False)  

In [0]:
import pandas as pd
import numpy as np
import os
import h5py
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def read_image_and_breast_mask(h5_filename):
    # 'r' mode is very important ! 
    with h5py.File(h5_filename, "r") as h5_file:
        image = h5_file["image"][:]
        breast_mask = h5_file["mask"][:]

    return image, breast_mask

In [0]:
def batch_generator(Train_df,batch_size,steps):
 idx=1
 while True: 
  yield load_data(Train_df,idx-1,batch_size)## Yields data
  if idx<steps:
    idx+=1
  else:
    idx=1

In [0]:
TRAIN_DIR = "breast_density_dataset/train/"
TEST_DIR = "breast_density_dataset/test/"

def load_data(Train_df,idx,batch_size):
 df = pd.read_csv("breast_density_dataset/train.csv")
 arr = np.arange(len(df))
 np.random.shuffle(arr)
 x = []
 z = []
 y = [] 
 for  row  in df.values[arr[0:batch_size],:]:
  image_uid=row[0]
  image_file = os.path.join(TRAIN_DIR, image_uid + ".h5")
  X, breast_mask = read_image_and_breast_mask(image_file)

  image = (X - X.min()) / (X.max() - X.min())
  #image =(image*255.).astype('uint8')
  
 # tmp=np.zeros(2)
 # tmp[row[-2]-1]=1
  #latent=np.array(functors([image.reshape(1,288, 208, 1), 1.])).reshape(36, 26, 128)
  #z.append(row[2:-1])
  x.append(image) 
  y.append(row[2:] )

 return (np.array(x).reshape(-1,288,208,1), np.array(y).reshape(-1,5))

In [0]:
import numpy as np
batch_size=32
steps_per_epoch=np.ceil(7960/batch_size)
validation_steps=np.ceil(2000/batch_size)

my_training_batch_generator = batch_generator('train.csv', 32,steps_per_epoch)
my_validation_batch_generator = batch_generator('valid.csv', 32,validation_steps)

In [0]:
x,y=next(my_training_batch_generator) 

In [0]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten,Reshape,Convolution2D,Cropping2D,Dropout
from keras.models import Model
from skimage.transform import resize
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau



input_img = Input(shape=(288,208,1))  # adapt this if using `channels_first` image data format

x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), padding='same')(x)



x=Flatten()(x)
x = Dense(1024)(x)

x=Dropout(rate=0.4)(x)

x = Dense(512)(x)

x=Dropout(rate=0.4)(x)

x = Dense(256)(x)

x=Dropout(rate=0.4)(x)

output = Dense(5,activation='relu')(x)


 



model = Model(input_img, [output])
model.compile(optimizer='adam', loss=['mse'])
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 288, 208, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 288, 208, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 288, 208, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 144, 104, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 144, 104, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 144, 104, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 72, 52, 128)       0   

In [0]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-s.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [0]:
results = model.fit_generator(my_training_batch_generator,epochs=30,steps_per_epoch=steps_per_epoch,verbose=1, validation_data=my_validation_batch_generator,validation_steps=validation_steps, callbacks=callbacks)

Epoch 1/30
249/249 [==============================] - 75s 302ms/step - loss: 0.0742 - val_loss: 0.0443

Epoch 00001: val_loss improved from 0.04497 to 0.04426, saving model to model-s.h5
Epoch 2/30
249/249 [==============================] - 75s 302ms/step - loss: 0.0724 - val_loss: 0.0411

Epoch 00002: val_loss improved from 0.04426 to 0.04106, saving model to model-s.h5
Epoch 3/30
249/249 [==============================] - 75s 301ms/step - loss: 0.0694 - val_loss: 0.0394

Epoch 00003: val_loss improved from 0.04106 to 0.03943, saving model to model-s.h5
Epoch 4/30
249/249 [==============================] - 75s 301ms/step - loss: 0.0694 - val_loss: 0.0408

Epoch 00004: val_loss did not improve from 0.03943
Epoch 5/30
249/249 [==============================] - 75s 301ms/step - loss: 0.0645 - val_loss: 0.0362

Epoch 00005: val_loss improved from 0.03943 to 0.03619, saving model to model-s.h5
Epoch 6/30
249/249 [==============================] - 75s 301ms/step - loss: 0.0643 - val_loss: 0

In [0]:
model.load_weights('model-s.h5')

In [0]:
df = pd.read_csv("breast_density_dataset/train.csv")
s=0
for  row  in df.values:
  image_uid=row[0]
  image_file = os.path.join(TRAIN_DIR, image_uid + ".h5")
  X, breast_mask = read_image_and_breast_mask(image_file)
  image = (X - X.min()) / (X.max() - X.min())
  tmp=model.predict(image.reshape(1,288, 208, 1))
  p=np.abs(np.round(tmp.reshape(-1)))
  if p[-1]==row[-1]:
    s+=1
print("precision :"+str(s/len(df)))

In [0]:
test = pd.read_csv("breast_density_dataset/test.csv")
TRAIN_DIR="breast_density_dataset/test"
y_pred=[]
for  row  in test.values:
  image_uid=row[0]
  image_file = os.path.join(TEST_DIR, image_uid + ".h5")
  X, breast_mask = read_image_and_breast_mask(image_file)
  image = (X - X.min()) / (X.max() - X.min())
  tmp=model.predict(image.reshape(1,288, 208, 1))
  p=np.abs(np.round(tmp.reshape(-1)))
  y_pred.append(int(p[-1]))

test["breast_density_score"]=y_pred

In [0]:
test.head()

,image_uid,study_uid,source,manufacturer,view,laterality,breast_density_score
0,bKoUoJrTDD,rvVNQjBYXRDgWGZabWmB,ANI,C,CC,L,1
1,jIvyWDKyKo,rvVNQjBYXRDgWGZabWmB,ANI,C,MLO,L,1
2,YLYzzzqCws,rvVNQjBYXRDgWGZabWmB,ANI,C,CC,R,2
3,kxORhMlINs,rvVNQjBYXRDgWGZabWmB,ANI,C,MLO,R,2
4,tesbnsFvoZ,xXGLZUFkRfyaIdjDlUkN,ANI,B,CC,L,3


In [0]:
test.to_csv("submissions.csv",index=False)